In [3]:
using LinearAlgebra, Symbolics

In [4]:
@variables r s;

In [5]:
ν = 0.3 # Coeficiente de Poisson
EM = 2.1E06 # Módulo de Young - kgf/cm^2
C = (EM/(1-ν^2)) * [1 ν 0; ν 1 0; 0 0 (1-ν)/2]; # Matriz de constantes elásticas (Tensión plana)

In [6]:
NodalCoords = [0 0; 3 0; 1.5 3] # Coordenadas Globales para elemento triangular de 3 nodos

3×2 Matrix{Float64}:
 0.0  0.0
 3.0  0.0
 1.5  3.0

In [7]:
# Funciones de interpolación para un elemento isoparamétrico triangular de 3 nodos
h1 = 1-r-s
h2 = r
h3 = s;

In [8]:
Dr = Differential(r)
Ds = Differential(s)

(::Differential) (generic function with 3 methods)

In [9]:
H = [h1 h2 h3]
∇H = [transpose([expand_derivatives.(Dr(H[i])) for i in 1:3]); transpose([expand_derivatives.(Ds(H[i])) for i in 1:3])]

2×3 Matrix{Num}:
 -1  1  0
 -1  0  1

In [11]:
x = expand((H * NodalCoords[:,1])[1])
y = expand((H * NodalCoords[:,2])[1])

3.0r + 1.5s
3.0s


In [12]:
J = expand.(∇H * NodalCoords) # Fundamentals of FEA, Koutromanos-pag.240 (ec.8.5.25)

J_inv = inv(J) # Inversa del Jacobiano
J_det = det(J) # Determinante del Jacobiano
display(J)
display(J_inv)
display(J_det)

2×2 Matrix{Num}:
 3.0  0.0
 1.5  3.0

2×2 Matrix{Num}:
  0.333333  -0.0
 -0.166667   0.333333

9.0

In [13]:
B1 = J_inv * ∇H
display(B1)

B = [B1[1,1] 0 B1[1,2] 0 B1[1,3] 0 
    0 B1[2,1] 0 B1[2,2] 0 B1[2,3] 
    B1[2,1] B1[1,1] B1[2,2] B1[1,2] B1[2,3] B1[1,3]]

2×3 Matrix{Num}:
 -0.333333   0.333333  0.0
 -0.166667  -0.166667  0.333333

3×6 Matrix{Num}:
 -0.333333   0          0.333333   0         0.0       0
  0         -0.166667   0         -0.166667  0         0.333333
 -0.166667  -0.333333  -0.166667   0.333333  0.333333  0.0

In [14]:
#Integramos por medio de cuadratura de Gauss para tres puntos sobre dominios triangulares
GL3T = [1/6 1/6; 2/3 1/6; 1/3 2/3] # Puntos de Gauss para integración
WGL3T = [1/3 1/3 1/3]

1×3 Matrix{Float64}:
 0.333333  0.333333  0.333333

In [15]:
K = zeros(6,6) 

# En este caso, el elemento posee deformación constante por lo que J, la inversa y el determinante, 
# junto con B son constantes
for i in 1:3                           
        K += 1/2 * (transpose(B) * C * B * J_det * WGL3T[i])        
    end
display(K)

6×6 Matrix{Num}:
      1.25481e6  375000.0             -1.05288e6  …  -2.01923e5  -3.46154e5
 375000.0             6.92308e5    28846.2           -4.03846e5  -5.76923e5
     -1.05288e6   28846.2              1.25481e6     -2.01923e5   3.46154e5
 -28846.2            -1.15385e5  -375000.0            4.03846e5  -5.76923e5
     -2.01923e5      -4.03846e5       -2.01923e5      4.03846e5   0.0
     -3.46154e5      -5.76923e5        3.46154e5  …   0.0         1.15385e6

In [16]:
 # Imponemos condiciones de contorno de Dirichlet
 # U1y=0 => U2=0
    K[2,:] .= 0
    K[:,2] .= 0
    K[2,2] = 1
 # U2x=0 => U3=0
    K[3,:] .= 0
    K[:,3] .= 0
    K[3,3] = 1
 # U2y=0 => U4=0
    K[4,:] .= 0
    K[:,4] .= 0
    K[4,4] = 1

display(K)

6×6 Matrix{Num}:
  1.25481e6  0  0  0  -2.01923e5  -3.46154e5
  0          1  0  0   0           0
  0          0  1  0   0           0
  0          0  0  1   0           0
 -2.01923e5  0  0  0   4.03846e5   0.0
 -3.46154e5  0  0  0   0.0         1.15385e6

In [17]:
#= \(A,B)
Matrix division using a polyalgorithm. For input matrices A and B, the result X is such that A*X == B when A is square. 
The solver that is used depends upon the structure of A. If A is upper or lower triangular (or diagonal), no factorization 
of A is required and the system is solved with either forward or backward substitution. For non-triangular square matrices, 
an LU factorization is used.
=#

In [21]:
# CASO 1. F1 = 500kgf ; F2 = 0kgf

R1 =[0, 0, 0, 0, 500., 0] # Fuerzas nodales en kgf.
U1 = K \ R1 # U1 --> Desplazamientos nodales.

E1 = B * U1 # Voigt notation
σ1 = C * E1
for i in 1:3
    if (abs(E1[i]) <= 1E-8)  E1[i] = 0 end
    if (abs(σ1[i]) <= 1E-8)  σ1[i] = 0 end
end
display("Desplazamientos nodales")
display(U1)
display("Deformaciones nodales")
display(E1)
display("Esfuerzos nodales")
display(σ1)

"Desplazamientos nodales"

6-element Vector{Num}:
 0.0002380952380952381
 0.0
 0.0
 0.0
 0.0013571428571428575
 7.142857142857141e-5

"Deformaciones nodales"

3-element Vector{Num}:
 -7.936507936507937e-5
  2.38095238095238e-5
  0.0004126984126984128

"Esfuerzos nodales"

3-element Vector{Num}:
 -166.66666666666666
    0
  333.33333333333337

In [19]:
# CASO 2. F1 = 0kgf ; F2 = 500kgf

R2 =[0, 0, 0, 0, 0, 500] # Fuerzas nodales en kgf.
U2 = K \ R2 # U2 --> Desplazamientos nodales.
display(U2)
E2 = B * U2
σ2 = C * E2
for i in 1:3
    if (abs(E2[i]) <= 1E-8)  E2[i] = 0 end
    if (abs(σ2[i]) <= 1E-8)  σ2[i] = 0 end
end
display(E2)
display(σ2)

6-element Vector{Num}:
 0.0001428571428571428
 0.0
 0.0
 0.0
 7.14285714285714e-5
 0.0004761904761904762

3-element Vector{Num}:
 -4.761904761904759e-5
  0.00015873015873015873
  0

3-element Vector{Num}:
   0
 333.3333333333333
   0

In [20]:
# CASO 3. F1 = 500kgf ; F2 = 500kgf

R3 =[0, 0, 0, 0, 500, 500] # Fuerzas nodales en kgf.
U3 = K \ R3 # U3 --> Desplazamientos nodales.
display(U3)
E3 = B * U3
σ3 = C * E3
for i in 1:3
    if (abs(E3[i]) <= 1E-8)  E3[i] = 0 end
    if (abs(σ3[i]) <= 1E-8)  σ3[i] = 0 end
end
display(E3)
display(σ3)

6-element Vector{Num}:
 0.0003809523809523809
 0.0
 0.0
 0.0
 0.001428571428571429
 0.0005476190476190477

3-element Vector{Num}:
 -0.00012698412698412695
  0.00018253968253968255
  0.00041269841269841285

3-element Vector{Num}:
 -166.6666666666666
  333.33333333333337
  333.3333333333334